### ChromaDB 설치

In [ ]:
pip install chromadb

In [ ]:
# `sentence_transformers` 필요 시 설치
#pip install sentence_transformers

### 통합할 ChromaDB 업로드

**반드시 해당 Chroma DB는 모든 파일이 온전한 상태로 있어야 DB 병합이 정상적으로 이루어짐**

In [2]:
from chromadb import PersistentClient

db1 = PersistentClient(path="/Users/kimss/Documents/place_v2/v2/chroma_db_merged_ver2")
db2 = PersistentClient(path="/Users/kimss/Documents/misson_gen_2/chroma_db")

#### 컬렉션 확인

In [3]:
print("📂 DB1에 있는 컬렉션:", [c.name for c in db1.list_collections()])
print("📂 DB2에 있는 컬렉션:", [c.name for c in db2.list_collections()])

📂 DB1에 있는 컬렉션: ['review_collection', 'mbti_traits', 'user_history', 'menu_collection', 'mbti_feeds', 'user_latest', 'recommendation_history', 'hobby_subtraits']
📂 DB2에 있는 컬렉션: ['hated_mission_collection', 'mission_collection']


#### 컬렉션 이름 리스트화

In [4]:
c1, c2 = [c.name for c in db1.list_collections()], [c.name for c in db2.list_collections()]

In [5]:
merged_db = PersistentClient(path="./chroma_db_merged")

# 복사할 컬렉션 이름들
collections_to_copy = []
arr = [c1, c2]
db_list = [db1, db2]

for i in range(len(arr)):
    for arr_collection in arr[i]:
        collections_to_copy.append((arr_collection, db_list[i]))

In [6]:
collections_to_copy

[('review_collection', <chromadb.api.client.Client at 0x10828cb90>),
 ('mbti_traits', <chromadb.api.client.Client at 0x10828cb90>),
 ('user_history', <chromadb.api.client.Client at 0x10828cb90>),
 ('menu_collection', <chromadb.api.client.Client at 0x10828cb90>),
 ('mbti_feeds', <chromadb.api.client.Client at 0x10828cb90>),
 ('user_latest', <chromadb.api.client.Client at 0x10828cb90>),
 ('recommendation_history', <chromadb.api.client.Client at 0x10828cb90>),
 ('hobby_subtraits', <chromadb.api.client.Client at 0x10828cb90>),
 ('hated_mission_collection', <chromadb.api.client.Client at 0x10e46d190>),
 ('mission_collection', <chromadb.api.client.Client at 0x10e46d190>)]

### 배치 단위로 통합 Chroma DB에 업로드

In [7]:
BATCH_SIZE = 4000

for col_name, src_db in collections_to_copy:
    print(f"복사 중: {col_name}")

    src_col = src_db.get_collection(col_name)
    dest_col = merged_db.get_or_create_collection(name=col_name)

    offset = 0
    total_copied = 0

    while True:
        data = src_col.get(
            limit=BATCH_SIZE,
            offset=offset,
            include=["embeddings", "documents", "metadatas"]
        )

        if not data["ids"]:
            break  # 더 이상 불러올 데이터 없음 → 루프 종료

        n = len(data["ids"])
        total_copied += n

        # ID 중복 방지를 위해 접두사 추가
        prefixed_ids = [f"{col_name}__{id}" for id in data["ids"]]

        if not (n == len(data["embeddings"]) == len(data["documents"]) == len(data["metadatas"])):
            raise ValueError(
                f"길이 불일치 발생 in {col_name}:\n"
                f"    ids({n}), emb({len(data['embeddings'])}), "
                f"doc({len(data['documents'])}), meta({len(data['metadatas'])})"
            )

        dest_col.add(
            ids=prefixed_ids,
            embeddings=data["embeddings"],
            documents=data["documents"],
            metadatas=data["metadatas"]
        )

        print(f"  └─ {offset}~{offset + n}번 항목 복사 완료")

        offset += n  # 다음 배치를 위해 오프셋 증가

    print(f"{col_name} 전체 복사 완료 → 총 {total_copied}개")
    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

복사 중: review_collection
  └─ 0~1243번 항목 복사 완료
review_collection 전체 복사 완료 → 총 1243개
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
복사 중: mbti_traits
  └─ 0~4000번 항목 복사 완료
  └─ 4000~6528번 항목 복사 완료
mbti_traits 전체 복사 완료 → 총 6528개
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
복사 중: user_history
  └─ 0~5번 항목 복사 완료
user_history 전체 복사 완료 → 총 5개
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
복사 중: menu_collection
  └─ 0~1243번 항목 복사 완료
menu_collection 전체 복사 완료 → 총 1243개
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
복사 중: mbti_feeds
  └─ 0~4000번 항목 복사 완료
  └─ 4000~8000번 항목 복사 완료
  └─ 8000~12000번 항목 복사 완료
  └─ 12000~16000번 항목 복사 완료
  └─ 16000~20000번 항목 복사 완료
  └─ 20000~24000번 항목 복사 완료
  └─ 24000~28000번 항목 복사 완료
  └─ 28000~31069번 항목 복사 완료
mbti_feeds 전체 복사 완료 → 총 31069개
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
복사 중: user_latest
  └─ 0~5번 항목 복사 완료
user_latest 전체 복사 완료 → 총 5개
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
복사 중: recommendation_history
recommendation_history 전체 복사 완료 → 총 0개
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
복사 중: hobby_s

### 통합 완료 여부 테스트

In [8]:
print("\n복사 완료 후 컬렉션 검증 시작")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

for col_name, _ in collections_to_copy:
    dest_col = merged_db.get_collection(col_name)
    count = dest_col.count()
    print(f"📁 {col_name} → 총 {count}개 저장됨")

    # 일부 샘플 확인 (선택)
    sample = dest_col.get(limit=1, include=["documents", "metadatas"])
    try:
        print(f"  └ 샘플 ID: {sample['ids'][0]}")
        print(f"  └ 샘플 문서 요약: {str(sample['documents'][0])[:60]}...")
        print(f"  └ 샘플 메타데이터: {sample['metadatas'][0]}")
    except:
        print('데이터 없음')


복사 완료 후 컬렉션 검증 시작
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📁 review_collection → 총 1243개 저장됨
  └ 샘플 ID: review_collection__review_collection__store_0
  └ 샘플 문서 요약: 알레그리아 판교테크노밸리점...
  └ 샘플 메타데이터: {'상호명': '알레그리아 판교테크노밸리점', '영업시간': '월, 목, 토, 일: 10:00~20:30, 화, 수, 금: 08:00~20:30', '주소': '경기 성남시 분당구 판교역로 230 삼환하이펙스 B동 1층 119호', '대표카테고리': '카페/디저트', '링크': 'https://place.map.kakao.com/18214074', '평균별점': 4.2, '경도': 127.11045570879, '위도': 37.4012850380545}
📁 mbti_traits → 총 6528개 저장됨
  └ 샘플 ID: mbti_traits__mbti_traits__933a5c7c-3750-464a-8820-13de29e48eca
  └ 샘플 문서 요약: 창의적...
  └ 샘플 메타데이터: {'MBTI': 'ENTP', 'Weight': 1.0, 'Trait': '창의적'}
📁 user_history → 총 5개 저장됨
  └ 샘플 ID: user_history__user_history__06c7d333-8bfc-4252-ba39-0d0846148fec
  └ 샘플 문서 요약: ...
  └ 샘플 메타데이터: {'user_id': 1, 'tf_score': 50, 'jp_score': 50, 'ei_score': 50, 'sn_score': 50, 'timestamp': '2025-05-15T14:09:55.484399', 'hobby_name': '코딩'}
📁 menu_collection → 총 1243개 저장됨
  └ 샘플 ID: menu_collection__menu_collection__store_0
  └